In [5]:
# Common
import tensorflow as tf
from glob import glob
import numpy as np

# Data
from sklearn.model_selection import train_test_split


# Data visualization
import matplotlib.pyplot as plt

# Model
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from keras.optimizers import Adam

# Metrics
from tensorflow.keras.metrics import *
import pathlib
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import load_model
from keras.optimizers import Adam

In [6]:
images = []
masks = []

masks_dir = r"C:\Users\Brodzio\OneDrive - Opolska eSzkola\Pulpit\AOM_Projekt\0013"
img_dir = r"C:\Users\Brodzio\OneDrive - Opolska eSzkola\Pulpit\AOM_Projekt\0013i"

# Wczytywanie masek
for filename in os.listdir(masks_dir):
    mask = cv2.imread(os.path.join(masks_dir, filename), 0)  # Wczytywanie maski w skali szarości
    mask = mask / 255.0  # Normalizacja wartości pikseli do zakresu 0-1
    mask = cv2.resize(mask, (256, 256))  # Zmiana rozmiaru maski na 256x256 pikseli
    mask = np.expand_dims(mask, axis=-1)  # Dodanie wymiaru osi do maski
    print(mask.shape)
    masks.append(mask)

# Wczytywanie obrazów
for filename in os.listdir(img_dir):
    img = cv2.imread(os.path.join(img_dir, filename))  # Wczytywanie obrazu
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Konwertowanie na skalę szarości
    img = img / 255.0  # Normalizacja wartości pikseli do zakresu 0-1
    img = cv2.resize(img, (256, 256))  # Zmiana rozmiaru obrazu na 256x256 pikseli
    img = np.expand_dims(img, axis=-1)  # Dodanie wymiaru osi do obrazu
    print(img.shape)
    images.append(img)

# Konwertowanie list na tablice numpy
images = np.array(images)
masks = np.array(masks)

(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 256, 1)
(256, 

In [7]:
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.1)